In [12]:
import os
import numpy as np

import data_preprocessing
import neural_metrics

In [26]:
data_str = 'openimage'
binary_num_epochs = 4
binary_model_name = 'dinov2_vits14'

preprocessor = data_preprocessing.DataPreprocessor(data_str=data_str)

binary_results_path = 'binary_results'
binary_l_strs = {f
                 for f in os.listdir(binary_results_path)
                 if f.startswith(f'{data_str}_{binary_model_name}')}
len(binary_l_strs)

In [27]:
for name in binary_l_strs:
    curr_path = fr'{binary_results_path}/{name}'
    curr_data = np.load(curr_path)
    l_str = name.split(f'e{binary_num_epochs - 1}_')[-1].replace('.npy', '')
    is_fine = l_str in preprocessor.fine_grain_classes_str
    l_str_index = getattr(preprocessor, 
                          'fine_grain_classes_str' if is_fine else 'coarse_grain_classes_str').index(l_str) 
    test = len(curr_data) == len(preprocessor.test_true_coarse_data)
    true_values = (preprocessor.test_true_fine_data 
                   if is_fine else preprocessor.test_true_coarse_data) \
        if test else (preprocessor.train_true_fine_data 
                      if is_fine else preprocessor.train_true_coarse_data)
    binary_true = np.where(true_values == l_str_index, 1, 0)
    accuracy, f1, precision, recall = neural_metrics.get_individual_metrics(pred_data=curr_data,
                                                                            true_data=binary_true)
    print(f'class: {l_str}; '
          f'accuracy: {accuracy:.2f}, '
          f'f1: {f1:.2f}, '
          f'precision: {precision:.2f}, '
          f'recall: {recall:.2f}')
    
    assert f1 > 0.7